In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os
import glob
from scipy import optimize
import importlib

In [2]:
project_path = os.getcwd()

## load_data

### get_latest_position_file

In [3]:
data_dir = f'{project_path}/data'

In [4]:
files = glob.glob(os.path.join(data_dir, 'Portfolio_Positions_*.csv'))
files

['/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-06-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Aug-05-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Nov-16-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-29-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Apr-02-2025.csv',
 '/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_May-06-2025.csv']

In [5]:
latest_file = None
latest_date = None

for f in files:
    basename = os.path.basename(f)
    date_part = basename.replace('Portfolio_Positions_', '').replace('.csv', '')
    try:
        date_obj = datetime.strptime(date_part, '%b-%d-%Y')
        if latest_date is None or date_obj > latest_date:
            latest_date = date_obj
            latest_file = f
    except ValueError:
        continue
[latest_file, latest_date]

['/Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv',
 datetime.datetime(2025, 12, 31, 0, 0)]

### clean_positions

In [6]:
from support_functions.analysis import get_latest_position_file, clean_currency

data_dir = f'{project_path}/data'
pos_file, pos_date = get_latest_position_file(data_dir)
positions_df = pd.read_csv(pos_file, index_col=False)

cols_to_clean = [
    'Last Price', 'Current Value', 
    'Cost Basis Total', 'Today\'s Gain/Loss Dollar', 
    'Total Gain/Loss Dollar'
]

In [7]:
for col in cols_to_clean:
    if col in positions_df.columns:
        positions_df[col] = positions_df[col].apply(clean_currency)

# Clean Quantity (remove match for formatting issues if any)
if 'Quantity' in positions_df.columns:
    positions_df['Quantity'] = pd.to_numeric(positions_df['Quantity'], errors='coerce').fillna(0)

### load_transactions

In [41]:
data_dir = f'{project_path}/data'
hist_files = glob.glob(os.path.join(data_dir, 'Accounts_History_*.csv'))
transactions_dfs = []
print(f"Found {len(hist_files)} history files.")

for f in hist_files:
    df = pd.read_csv(f, on_bad_lines='skip') 
    transactions_dfs.append(df)

transactions_df = pd.concat(transactions_dfs, ignore_index=True)

Found 4 history files.


In [44]:
[transactions_df.iloc[292]['Run Date'],transactions_df.iloc[228]['Run Date']]

['11/13/2025', ' 02/15/2024']

### clean_transactions

In [18]:
from support_functions import analysis
importlib.reload(analysis)
from support_functions.analysis import load_transactions, clean_currency
data_dir = f'{project_path}/data'
transactions_df = load_transactions(data_dir)

Found 4 history files.


In [19]:
for col in transactions_df.columns:
    if (
        (transactions_df[col].dtype == 'object' ) or 
        (transactions_df[col].dtype == 'string')
    ):
        transactions_df[col] = transactions_df[col].str.strip()   
# Standardize dates
transactions_df['Run Date'] = pd.to_datetime(transactions_df['Run Date'], errors='coerce')
# Sometimes 'Settlement Date' exists
if 'Settlement Date' in transactions_df.columns:
        transactions_df['Settlement Date'] = pd.to_datetime(transactions_df['Settlement Date'], errors='coerce')
    


# Clean numeric columns
hist_numeric_cols = [
'Amount ($)', 'Price ($)', 'Quantity', 
    'Commission ($)', 'Fees ($)', 'Accrued Interest ($)'
]
for col in hist_numeric_cols:
    if col in transactions_df.columns:
        transactions_df[col] = transactions_df[col].apply(clean_currency)
    
# Sort by date
transactions_df = transactions_df.sort_values('Run Date')

In [21]:
transactions_df[transactions_df['Symbol'] == 'AAPL']

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
537,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,9.0,164.61,0.0,0.0,0.0,-1481.45,2022-08-09
538,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,1.0,163.54,0.0,0.0,0.0,-163.54,2022-08-09
536,2022-08-26,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,164.50,0.0,0.0,0.0,-1645.00,2022-08-30
535,2022-08-29,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,160.00,0.0,0.0,0.0,-3200.00,2022-08-31
534,2022-11-01,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,150.00,0.0,0.0,0.0,-1500.00,2022-11-03
533,2022-11-08,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,138.50,0.0,0.0,0.0,-2770.00,2022-11-10
563,2022-11-10,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,11.50,NaT
629,2023-02-16,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.10,NaT
628,2023-05-18,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT
627,2023-08-17,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT


### load_data

In [22]:
from support_functions import analysis
importlib.reload(analysis)
from support_functions.analysis import (
    get_latest_position_file, clean_positions,
    load_transactions, clean_transactions
)

In [23]:
data_dir = f'{project_path}/data'
pos_file, pos_date = get_latest_position_file(data_dir)
print(f"Loading positions from: {pos_file} (Date: {pos_date.strftime('%Y-%m-%d')})")
positions_df = pd.read_csv(pos_file, index_col=False)

positions_df = clean_positions(positions_df)


Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)


In [24]:
transactions_df = load_transactions(data_dir)
    
transactions_df = clean_transactions(transactions_df)

Found 4 history files.


In [26]:
transactions_df[transactions_df['Symbol'] == 'AAPL']

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
537,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,9.0,164.61,0.0,0.0,0.0,-1481.45,2022-08-09
538,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,1.0,163.54,0.0,0.0,0.0,-163.54,2022-08-09
536,2022-08-26,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,164.50,0.0,0.0,0.0,-1645.00,2022-08-30
535,2022-08-29,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,160.00,0.0,0.0,0.0,-3200.00,2022-08-31
534,2022-11-01,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,150.00,0.0,0.0,0.0,-1500.00,2022-11-03
533,2022-11-08,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,138.50,0.0,0.0,0.0,-2770.00,2022-11-10
563,2022-11-10,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,11.50,NaT
629,2023-02-16,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.10,NaT
628,2023-05-18,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT
627,2023-08-17,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT


## analyze_symbol_performance

### categorize_asset

In [8]:
from support_functions.analysis import load_data

In [9]:
data_dir = f'{project_path}/data'
positions_df, history_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [10]:
i = 10
row = positions_df.loc[i]
print(row)

Account Number                Z23390746
Account Name                 Individual
Symbol                             AAPL
Description                   APPLE INC
Quantity                           70.0
Last Price                       271.86
Last Price Change                -$1.22
Current Value                   19030.2
Today's Gain/Loss Dollar          -85.4
Today's Gain/Loss Percent        -0.45%
Total Gain/Loss Dollar          8270.21
Total Gain/Loss Percent         +76.86%
Percent Of Account                1.52%
Cost Basis Total               10759.99
Average Cost Basis              $153.71
Type                               Cash
Name: 10, dtype: object


In [11]:
symbol = str(row['Symbol'])
desc = str(row.get('Description', '')).upper()
mmf_symbols = ['FZFXX', 'FDRXX', 'SPAXX', 'QUSBQ'] 
symbol in mmf_symbols or 'MONEY MARKET' in desc or 'CASH RESERVES' in desc or 'FDIC INSURED DEPOSIT' in desc
(len(symbol) >= 8 and symbol.startswith('912')) or 'TREAS BILL' in desc or 'TREASURY BILL' in desc

False

### xirr

In [13]:
from support_functions.analysis import (
    load_data,categorize_asset
)
data_dir = f'{project_path}/data'
positions_df, history_df, latest_date = load_data(data_dir)

Loading positions from: /Users/yifanli/Github/fidelity-portfolio-tracker/data/Portfolio_Positions_Dec-31-2025.csv (Date: 2025-12-31)
Found 4 history files.


In [14]:
i = 10
row = positions_df.loc[i]
print(row)

Account Number                Z23390746
Account Name                 Individual
Symbol                             AAPL
Description                   APPLE INC
Quantity                           70.0
Last Price                       271.86
Last Price Change                -$1.22
Current Value                   19030.2
Today's Gain/Loss Dollar          -85.4
Today's Gain/Loss Percent        -0.45%
Total Gain/Loss Dollar          8270.21
Total Gain/Loss Percent         +76.86%
Percent Of Account                1.52%
Cost Basis Total               10759.99
Average Cost Basis              $153.71
Type                               Cash
Name: 10, dtype: object


In [23]:
symbol = row['Symbol']
account_num = row['Account Number']
account_name = row['Account Name']
current_val = row['Current Value']
quantity = row['Quantity']
asset_type = categorize_asset(row)

# Filter History
mask = (history_df['Account Number'] == account_num) & (history_df['Symbol'] == symbol)
symbol_hist = history_df[mask]
symbol_hist

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
538,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,1.0,163.54,0.0,0.0,0.0,-163.54,2022-08-09
537,2022-08-05,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,9.0,164.61,0.0,0.0,0.0,-1481.45,2022-08-09
536,2022-08-26,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,164.50,0.0,0.0,0.0,-1645.00,2022-08-30
535,2022-08-29,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,160.00,0.0,0.0,0.0,-3200.00,2022-08-31
534,2022-11-01,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,10.0,150.00,0.0,0.0,0.0,-1500.00,2022-11-03
533,2022-11-08,Individual,Z23390746,YOU BOUGHT APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,20.0,138.50,0.0,0.0,0.0,-2770.00,2022-11-10
563,2022-11-10,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,11.50,NaT
629,2023-02-16,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.10,NaT
628,2023-05-18,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT
627,2023-08-17,Individual,Z23390746,DIVIDEND RECEIVED APPLE INC (AAPL) (Cash),AAPL,APPLE INC,Cash,0.0,0.00,0.0,0.0,0.0,16.80,NaT


In [22]:
cash_flows = []
relevant_actions = symbol_hist[symbol_hist['Amount ($)'].notna()].reset_index(drop=True)
total_invested = 0
total_returned = 0
for _, h_row in relevant_actions.iterrows():
    date = h_row['Run Date']
    amt = h_row['Amount ($)']
    cash_flows.append((date, amt))
    
    if amt < 0:
        total_invested += abs(amt)
    else:
        total_returned += amt
if current_val > 0:
    cash_flows.append((latest_date, current_val))
cash_flows

[(Timestamp('2022-08-05 00:00:00'), -163.54),
 (Timestamp('2022-08-05 00:00:00'), -1481.45),
 (Timestamp('2022-08-26 00:00:00'), -1645.0),
 (Timestamp('2022-08-29 00:00:00'), -3200.0),
 (Timestamp('2022-11-01 00:00:00'), -1500.0),
 (Timestamp('2022-11-08 00:00:00'), -2770.0),
 (Timestamp('2022-11-10 00:00:00'), 11.5),
 (Timestamp('2023-02-16 00:00:00'), 16.1),
 (Timestamp('2023-05-18 00:00:00'), 16.8),
 (Timestamp('2023-08-17 00:00:00'), 16.8),
 (Timestamp('2023-11-16 00:00:00'), 16.8),
 (Timestamp('2024-02-15 00:00:00'), 16.8),
 (Timestamp('2024-05-16 00:00:00'), 17.5),
 (Timestamp('2024-08-15 00:00:00'), 17.5),
 (Timestamp('2024-11-14 00:00:00'), 17.5),
 (NaT, 18.2),
 (NaT, 18.2),
 (NaT, 18.2),
 (NaT, 17.5),
 (datetime.datetime(2025, 12, 31, 0, 0), np.float64(19030.2))]

[(Timestamp('2022-08-05 00:00:00'), -163.54),
 (Timestamp('2022-08-05 00:00:00'), -1481.45),
 (Timestamp('2022-08-26 00:00:00'), -1645.0),
 (Timestamp('2022-08-29 00:00:00'), -3200.0),
 (Timestamp('2022-11-01 00:00:00'), -1500.0),
 (Timestamp('2022-11-08 00:00:00'), -2770.0),
 (Timestamp('2022-11-10 00:00:00'), 11.5),
 (Timestamp('2023-02-16 00:00:00'), 16.1),
 (Timestamp('2023-05-18 00:00:00'), 16.8),
 (Timestamp('2023-08-17 00:00:00'), 16.8),
 (Timestamp('2023-11-16 00:00:00'), 16.8),
 (Timestamp('2024-02-15 00:00:00'), 16.8),
 (Timestamp('2024-05-16 00:00:00'), 17.5),
 (Timestamp('2024-08-15 00:00:00'), 17.5),
 (Timestamp('2024-11-14 00:00:00'), 17.5),
 (NaT, 18.2),
 (NaT, 18.2),
 (NaT, 18.2),
 (NaT, 17.5)]

[(Timestamp('2022-08-05 00:00:00'), -163.54),
 (Timestamp('2022-08-05 00:00:00'), -1481.45),
 (Timestamp('2022-08-26 00:00:00'), -1645.0),
 (Timestamp('2022-08-29 00:00:00'), -3200.0),
 (Timestamp('2022-11-01 00:00:00'), -1500.0),
 (Timestamp('2022-11-08 00:00:00'), -2770.0),
 (Timestamp('2022-11-10 00:00:00'), 11.5),
 (Timestamp('2023-02-16 00:00:00'), 16.1),
 (Timestamp('2023-05-18 00:00:00'), 16.8),
 (Timestamp('2023-08-17 00:00:00'), 16.8),
 (Timestamp('2023-11-16 00:00:00'), 16.8),
 (Timestamp('2024-02-15 00:00:00'), 16.8),
 (Timestamp('2024-05-16 00:00:00'), 17.5),
 (Timestamp('2024-08-15 00:00:00'), 17.5),
 (Timestamp('2024-11-14 00:00:00'), 17.5),
 (NaT, 18.2),
 (NaT, 18.2),
 (NaT, 18.2),
 (NaT, 17.5)]